In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data=tfds.load("imdb_reviews",as_supervised=True)

In [3]:
train_dataset,test_dataset=data["train"],data["test"]

In [4]:
batch_size=32
train_dataset=train_dataset.shuffle(10000)
train_dataset=train_dataset.batch(batch_size)
test_dataset=test_dataset.batch(batch_size)

In [5]:
exm,label=next(iter(train_dataset))
print("Text:\n",exm.numpy()[0])
print("\nlabel: ",label.numpy()[0])

Text:
 b'Not sure why this movie seems to have gotten such rave reviews.<br /><br />While watching "Bang" one night on TV, I found myself bored by the nonsensical, random plot which was occurring on screen. The entire movie seems to be nothing more than an exercise in meaningless, artsy-fartsy self-indulgence on the part of the filmmaker. The fact that the director/writer goes by a one name moniker only reinforces this sense of pretentiousness. <br /><br />Those interested in indie flicks would be better off looking for something better written and dare I say, more entertaining than this complete waste of time.'

label:  0


In [6]:
encoder=tf.keras.layers.TextVectorization(max_tokens=9000)
encoder.adapt(train_dataset.map(lambda text, _:text))

vocabulary=np.array(encoder.get_vocabulary())

original_text=exm.numpy()[0]
encoded_text=encoder(original_text).numpy()
decoded_text=" ".join(vocabulary[encoded_text])

print("original: ",original_text)
print("encoded: ",encoded_text)
print("decoded: ",decoded_text)

original:  b'Not sure why this movie seems to have gotten such rave reviews.<br /><br />While watching "Bang" one night on TV, I found myself bored by the nonsensical, random plot which was occurring on screen. The entire movie seems to be nothing more than an exercise in meaningless, artsy-fartsy self-indulgence on the part of the filmmaker. The fact that the director/writer goes by a one name moniker only reinforces this sense of pretentiousness. <br /><br />Those interested in indie flicks would be better off looking for something better written and dare I say, more entertaining than this complete waste of time.'
encoded:  [  22  243  134   11   18  181    6   26 1772  136 5152    1   13  131
  147 4745   29  311   21  244   10  249  532 1072   33    2 4720 1473
  114   60   14    1   21  293    2  417   18  181    6   28  158   52
   71   34 3416    8 4022    1    1   21    2  171    5    2 1565    2
  185   12    2    1  261   33    4   29  398    1   61    1   11  278
    5    1 

In [7]:
model=tf.keras.Sequential([
    encoder,tf.keras.layers.Embedding(
    len(encoder.get_vocabulary()),64,mask_zero=True),
    
    tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(64,return_sequences=True)),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, None)              0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, None, 64)          576000    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

In [8]:
model.compile(
       loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
     metrics=["accuracy"])

In [9]:
history=model.fit(train_dataset,epochs=4,validation_data=test_dataset)

Epoch 1/4



782/782 [==============================] - 1684s 2s/step - loss: 0.3776 - accuracy: 0.8203 - val_loss: 0.3690 - val_accuracy: 0.8101
Epoch 2/4
782/782 [==============================] - 2264s 3s/step - loss: 0.2253 - accuracy: 0.9072 - val_loss: 0.3032 - val_accuracy: 0.8690
Epoch 3/4
782/782 [==============================] - 1456s 2s/step - loss: 0.1489 - accuracy: 0.9436 - val_loss: 0.3569 - val_accuracy: 0.8428
Epoch 4/4
782/782 [==============================] - 1524s 2s/step - loss: 0.0905 - accuracy: 0.9659 - val_loss: 0.3986 - val_accuracy: 0.8502


In [16]:
predictions=model.predict(np.array(["12th fail movie is fantastic"]))
print(*predictions[0])

if predictions[0]>0:
    print("positive")
    
else:
    print("negative")

1/1 [==============================] - 0s 40ms/step
-1.5923638
negative
